In [1]:
!pip install -q -U sentence-transformers faiss-cpu peft huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.4/483.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import faiss
import torch
from pathlib import Path
from sentence_transformers import SentenceTransformer
from peft import PeftModel
from google.colab import drive
from huggingface_hub import hf_hub_download

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
BASE_MODEL = "Qwen/Qwen3-Embedding-0.6B"
ADAPTER_REPO = "Isaac74/qwen3-0.6b-lightweight-semantic-mathlib-search-adapter"
HNSW_IDX_FILENAME = "mathlib420_final_hnsw.idx"
PUBMAP_FILENAME   = "mathlib420_final_public_index.jsonl"

INDEX_PATH = Path(hf_hub_download(
    repo_id=ADAPTER_REPO,
    filename=f"faiss/{HNSW_IDX_FILENAME}",
    repo_type="model"
))
PUBMAP_PATH_JSONL = Path(hf_hub_download(
    repo_id=ADAPTER_REPO,
    filename=f"faiss/{PUBMAP_FILENAME}",
    repo_type="model"
))

# --- load FAISS index (kept) ---
index = faiss.read_index(str(INDEX_PATH))
index.hnsw.efSearch = 128

# --- load base encoder + LoRA adapter ---
model = SentenceTransformer(BASE_MODEL, trust_remote_code=True)
model.max_seq_length = 256
hf_model = model._first_module().auto_model
peft_model = PeftModel.from_pretrained(
    hf_model,
    ADAPTER_REPO,
    subfolder="adapter",
    is_trainable=False
)
model._first_module().auto_model = peft_model
model.eval()

# --- load public mapping ---
idx2meta = {}
with PUBMAP_PATH_JSONL.open("r", encoding="utf-8") as f:
    for line in f:
        if not line.strip():
            continue
        d = json.loads(line)
        idx2meta[int(d["idx"])] = (d["full_name"], d["statement"])

# --- warmup ---
_ = model.encode(["warmup"], convert_to_numpy=True, normalize_embeddings=True)
import numpy as np
dummy = np.zeros((1, index.d), dtype="float32")
index.search(dummy, 1)

print("Search ready")

mathlib420_final_hnsw.idx:   0%|          | 0.00/2.56G [00:00<?, ?B/s]

mathlib420_final_public_index.jsonl:   0%|          | 0.00/121M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/840 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/18.4M [00:00<?, ?B/s]

Search ready


In [7]:
import re

def normalize_math_symbols(text: str) -> str:
    replacements = {
        "<=": "≤", ">=": "≥", "!=": "≠",
        "=>": "⇒", "<=>": "⇔", "->": "⇒",
        "÷": "/"
    }
    for old, new in sorted(replacements.items(), key=lambda kv: -len(kv[0])):
        text = text.replace(old, new)
    text = re.sub(r"\bpi\b", " π ", text, flags=re.IGNORECASE)

    ops = sorted([
        "≤","≥","≠","⇒","→","←","⇔",
        "+","-","*","/","=","<",">",
        "⊂","⊆","⊄","⊇","⊃","∈","∉","∪","∩","⊥","∥","∅",
        "∀","∃","¬","~","∧","∨","±","√","∑","∏","≈","≅","≃","≡"
    ], key=len, reverse=True)
    pattern = re.compile(r'\s*(' + '|'.join(map(re.escape, ops)) + r')\s*')
    norm = pattern.sub(r' \1 ', text)
    return re.sub(r'\s{2,}', ' ', norm).strip()

def unique_top_k(query: str, k: int = 10, search_k: int = 50):
    q_norm = normalize_math_symbols(query)
    q_vec = model.encode([q_norm], convert_to_numpy=True, normalize_embeddings=True).astype("float32")
    D, I = index.search(q_vec, search_k)
    seen, out = set(), []
    for score, idx in zip(D[0], I[0]):
        meta = idx2meta.get(int(idx))
        if meta is None:
            continue
        full_name, stmt = meta
        if full_name in seen:
            continue
        seen.add(full_name)
        out.append((full_name, stmt, float(score)))
        if len(out) == k:
            break
    return out

# --- interactive search loop ---
print("\nEnter a natural language/formula-based search query (type 'exit' to quit)\n" + "-"*60)
while True:
    q = input("🔎  ")
    if q.strip().lower() in {"exit", "quit"}:
        break
    results = unique_top_k(q, k=10, search_k=50)
    for full_name, stmt, score in results:
        print(f"{full_name}\n{stmt}\n")
    print("-"*60)


Enter a natural language/formula-based search query (type 'exit' to quit)
------------------------------------------------------------
🔎  -w*-l=w*l
neg_mul_neg
theorem neg_mul_neg (a b : α) : -a * -b = a * b :=

ISize.neg_mul_neg
theorem ISize.neg_mul_neg (a b : ISize) : -a * -b = a * b :=

neg_smul_neg
theorem neg_smul_neg : -r • -x = r • x :=

neg_dotProduct_neg
lemma neg_dotProduct_neg : -v ⬝ᵥ -w = v ⬝ᵥ w :=

ContinuousMultilinearMap.neg_prod_neg
theorem neg_prod_neg [AddCommGroup M₃] [Module R M₃] [TopologicalSpace M₃]     [IsTopologicalAddGroup M₃] (f : ContinuousMultilinearMap R M₁ M₂)     (g : ContinuousMultilinearMap R M₁ M₃) : (-f).prod (-g) = - f.prod g :=

Int.neg_mul_neg
theorem neg_mul_neg (a b : Int) : -a * -b = a * b :=

MeasureTheory.Lp.neg_smul_neg
protected lemma neg_smul_neg (f : Lp 𝕜 p μ) (g : Lp E q μ) :     -f • -g = f • g :=

Int8.neg_mul_comm
theorem Int8.neg_mul_comm (a b : Int8) : -a * b = a * -b :=

neg_mul_comm
theorem neg_mul_comm (a b : α) : -a * b = a * 